# Data Visualization

In [ ]:
import pandas as pd
from pathlib import Path

xbd_path = 'datasets/xbd'
subsets = ('/train_bldgs/', '/hold_bldgs/', '/test_bldgs/', '/tier3_bldgs/')
disaster_folders = os.listdir(xbd_path + subsets[0])

i_subset = 0
i_disaster = 5

print(list(Path(xbd_path + subsets[i_subset] + disaster_folders[i_disaster]).glob('*.csv*'))[0])
labels = pd.read_csv(list(Path(xbd_path + subsets[i_subset] + disaster_folders[i_disaster]).glob('*.csv*'))[0])
labels.columns = ['name', 'xcoords', 'ycoords', 'long', 'lat', 'class']
zone = lambda row: '_'.join(row['name'].split('_', 2)[:2])
labels['zone'] = labels.apply(zone, axis=1)
labels['zone'].value_counts()
#labels['zone'].value_counts()[labels['zone'].value_counts()==1].index.tolist()

In [ ]:
from visualization import plot_on_image

plot_on_image(labels, subsets[i_subset], 'mexico-earthquake_00000192')

In [ ]:
from visualization import plot_on_map

plot_on_map(labels, mapbox=False)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from visualization import CmapString

cmap = CmapString(palette='viridis', domain=labels['zone'].values)

plt.figure(figsize=(12,8))
for _, row in labels.iterrows():
    plt.scatter(row['xcoords'], row['ycoords'], label=row['zone'], color=cmap.color(row['zone']))
plt.axis('off')
plt.show()

---

# Graph Generation

In [ ]:
import os
from pathlib import Path
import json
import pandas as pd

In [ ]:
path = 'datasets/xbd/hold_bldgs/'
disaster_folders = os.listdir(path)
disaster = 'mexico-earthquake'

labels = pd.read_csv(list(Path(path + disaster).glob('*.csv*'))[0], index_col=0)
labels.drop(columns=['long','lat'], inplace=True)
zone = lambda row: '_'.join(row.name.split('_', 2)[:2])
labels['zone'] = labels.apply(zone, axis=1)

processed_files = []
zones = labels['zone'].value_counts()[labels['zone'].value_counts()>1].index.tolist()
for zone in zones:
     if not ((labels[labels['zone'] == zone]['class'] == 'un-classified').all() or \
            (labels[labels['zone'] == zone]['class'] != 'un-classified').sum() == 1):
        processed_files.append(f'{zone}.pt')

len(processed_files)

---

# Training

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.data import GraphSAINTNodeSampler
from tqdm import tqdm
from dataset import xBD
from model import DeeperGCN
from metrics import xview2_f1_score

with open('exp_settings.json', 'r') as JSON:
    settings_dict = json.load(JSON)

seed = settings_dict['seed']
batch_size = settings_dict['data']['batch_size']
num_steps = settings_dict['data']['saint_num_steps']
name = settings_dict['model']['name']

train_roots = ["C:/thesis/datasets/xbd/mexico_train"]
test_roots = ["C:/thesis/datasets/xbd/mexico_test"]

mexico_hold = ["C:/thesis/datasets/xbd/mexico_hold"]

hidden_units = settings_dict['model']['hidden_units']
num_layers = settings_dict['model']['num_layers']
dropout_rate = settings_dict['model']['dropout_rate']
lr = settings_dict['model']['lr']
n_epochs = settings_dict['epochs']
starting_epoch = settings_dict['starting_epoch']
path = settings_dict['model']['path']
save_best_only = settings_dict['save_best_only']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
train_data_list = [xBD(train_roots[0], 'mexico-earthquake', 'train')]
#test_data_list = [xBD(test_roots[0], 'mexico-earthquake', 'test')]
#hold_data_list = [xBD(mexico_hold[0], 'mexico-earthquake', 'hold')]

In [ ]:
y_all = []
for dataset in train_data_list:
    y_all.extend([data.y for data in dataset])
y_all = torch.cat(y_all)

In [ ]:
from metrics import parse_ordinal_output
y_allp = parse_ordinal_output(y_all)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_allp), y=y_allp)

In [ ]:
dataset = train_data_list[0]
model = DeeperGCN(dataset.num_node_features, dataset.num_edge_features, hidden_units, dataset.num_classes,num_layers, dropout_rate)

In [ ]:
model.train()
data = dataset[0]
sampler = GraphSAINTNodeSampler(data, batch_size=batch_size, num_steps=num_steps, num_workers=2)

In [ ]:
for subdata in sampler:
    subdata = subdata.to(device)
    out = model(subdata.x, subdata.edge_index, subdata.edge_attr)
    loss = F.binary_cross_entropy(input=out, target=subdata.y.float(), weight=torch.Tensor(class_weights))
    print(loss)
    break